In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

plt.style.use('fivethirtyeight') 

import re

import json
import csv

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
with open('/kaggle/input/alqac-2023-training-data-v11/ALQAC_2023_training_data_v1.1/train.json', encoding="utf8") as f:
    train_2023_data = json.load(f)
with open('/kaggle/input/alqac-2023-training-data-v11/ALQAC_2023_training_data_v1.1/additional_data/zalo/zalo_question.json', encoding="utf8") as f:
    zalo_data = json.load(f)
with open('/kaggle/input/alqac-2023-training-data-v11/ALQAC_2023_training_data_v1.1/additional_data/ALQAC_2022_training_data/question.json', encoding="utf8") as f:
    train_2022_data = json.load(f)
with open('/kaggle/input/alqac-2023-training-data-v11/ALQAC_2023_training_data_v1.1/public_test.json', encoding="utf8") as f:
    test_2023_data = json.load(f)
with open('/kaggle/input/clause-contrastive/clauses_contrastive_2023.json', encoding="utf8") as f:
    contrastive_data = json.load(f)             
# contrastive_data = train_2023_data

In [3]:
data = []

for item in train_2023_data:
    query_1 = item['text']
    row = {
        'original_text': query_1
    }
    data.append(row)

for item in train_2022_data:
    query_2 = item['text']
    row = {
        'original_text': query_2
    }
    data.append(row)

for item in zalo_data:
    query_3 = item['text']
    row = {
        'original_text': query_3
    }
    data.append(row)

for item in test_2023_data:
    query_4 = item['text']
    row = {
        'original_text': query_4
    }
    data.append(row)

# Create a dataframe from the list of rows
df = pd.DataFrame(data)
df

,original_text
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ..."
4,"""Trọng tại thương mại"" là phương thức giải quy..."
...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...


In [4]:
df.isnull().sum()

original_text    0
dtype: int64

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Multi-domain :D, có data từ legal document 
# Link: https://github.com/vietai/mTet

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(50048, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50048, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Vi -> En and En -> Vi

In [15]:
def translate_vi2en(inputs):
    outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    updated_list = [text.lstrip("'en: ").rstrip("'") for text in res]
    updated_list = str(updated_list)
    updated_list = updated_list.lstrip("[''").rstrip("'']")
    return updated_list

In [16]:
def translate_en2vi(inputs):
    outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    updated_list = [text.lstrip("'vi: ").rstrip("'") for text in res]
    updated_list = str(updated_list)
    updated_list = updated_list.lstrip("[''").rstrip("'']")
    return updated_list

In [18]:
law_test = "Người nghiện ma túy từ đủ 18 tuổi trở lên đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật xử lý vi phạm hành chính khi bị phát hiện sử dụng trái phép chất ma túy trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?"
print(translate_vi2en(law_test))
print(type(translate_vi2en(law_test)))


Drug addicts aged full 18 years or older who are being applied administrative handling measure of sending to compulsory detoxification establishments as prescribed by the Law on handling of administrative violations when being detected to illegally use narcotic substances during the period of voluntary drug detoxification, right or wrong?
<class 'str'>


In [19]:
law_test2 = "Drug addicts aged full 18 years or older who are subject to the administrative handling measure of sending to compulsory detoxification establishments under the provisions of the Law on handling of administrative violations when they are detected to use narcotic substances illegally during the period of voluntary drug detoxification, right or wrong?"
print(translate_en2vi(law_test2))

Người nghiện ma túy từ đủ 18 tuổi trở lên đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật xử lý vi phạm hành chính khi bị phát hiện sử dụng trái phép chất ma túy trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?


In [20]:
df['vi2en'] = df['original_text'].apply(translate_vi2en)

In [21]:
df

,original_text,vi2en
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of..."
...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...


In [28]:
df['vi2en'][1022]

'What is the regulation of the list of areas to dredge seaport waters?'

In [29]:
df['en2vi'] = df['vi2en'].apply(translate_en2vi)

In [30]:
df

,original_text,vi2en,en2vi
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy..."
...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...


In [31]:
df['en2vi'][3913]

'Trong những điều kiện sau đây phải đáp ứng cho nam và nữ kết hôn với nhau?'

In [33]:
# df.to_csv('output.csv', index=False, encoding='utf-8-sig')

In [34]:
test_df = df 
test_df

,original_text,vi2en,en2vi
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy..."
...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...


In [36]:
count_different = (test_df['original_text'] != test_df['en2vi']).sum()
print(count_different)

3878


In [40]:
print(test_df['original_text'][500])
print(test_df['en2vi'][500])

Doanh nghiệp cung cấp dịch vụ mang cần làm gì để đảm bảo an toàn cho trẻ em?
Người vận chuyển cần phải làm gì để đảm bảo an toàn cho trẻ?


## Ja -> Vi and Vi -> Ja

In [42]:
# !pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
checkpoint = "facebook/nllb-200-distilled-600M"
# Lots of language but not really domain-specific :( 

model_2 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer_2 = AutoTokenizer.from_pretrained(checkpoint)

In [52]:
from transformers import pipeline

In [57]:
def translate_vi2ja(inputs):
    target_lang = 'jpn_Jpan'
    source_lang = 'vie_Latn'
    translator = pipeline('translation', model=model_2, tokenizer=tokenizer_2, src_lang=source_lang, tgt_lang=target_lang, max_length = 400)
    output = translator(inputs)
    translated_text = output[0]['translation_text']
    return translated_text

In [58]:
text_2 = "Người nghiện ma túy từ đủ 18 tuổi trở lên đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật xử lý vi phạm hành chính khi bị phát hiện sử dụng trái phép chất ma túy trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?"
print(translate_vi2ja(text_2))

18歳以上の薬物依存者が 薬物中毒期間中に 薬物使用が検出された場合 違法行為行為行為法に規定されている 薬物中毒施設に強制送還される行政処置措置が適用されている場合 正しいか間違っているか?


In [59]:
def translate_ja2vi(inputs):
    target_lang = 'vie_Latn'
    source_lang = 'jpn_Jpan'
    translator = pipeline('translation', model=model_2, tokenizer=tokenizer_2, src_lang=source_lang, tgt_lang=target_lang, max_length = 400)
    output = translator(inputs)
    translated_text = output[0]['translation_text']
    return translated_text

In [61]:
text_3 = "18歳以上の薬物依存者が 薬物中毒期間中に 薬物使用が検出された場合 違法行為行為行為法に規定されている 薬物中毒施設に強制送還される行政処置措置が適用されている場合 正しいか間違っているか?"
print(translate_ja2vi(text_3))

Nếu một người nghiện ma túy trên 18 tuổi được phát hiện sử dụng ma túy trong thời gian nghiện ma túy, liệu điều đó có đúng hay sai khi hành động hành pháp bắt buộc phải được đưa về nhà nghiện ma túy theo quy định của luật hành vi phạm tội không hợp pháp?


In [62]:
# Duma dich buon cuoi =)))

In [ ]:
test_df['vi2ja'] = test_df['original_text'].apply(translate_vi2ja)
test_df

## Backtranslation using MarianMT

### Engrisk

In [2]:
from transformers import MarianMTModel, MarianTokenizer

In [3]:
# Read the CSV file
test_df = pd.read_csv('/kaggle/input/saved-data-backtranslation/output (1).csv')
test_df

,original_text,vi2en,en2vi
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy..."
...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...


In [4]:
# vi -> en

# Get the name of the first model
first_model_name = 'Helsinki-NLP/opus-mt-vi-en'
# Get the tokenizer
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
# Load the pretrained model based on the name
first_model = MarianMTModel.from_pretrained(first_model_name)

In [5]:
# en -> vi

# Get the name of the second model
second_model_name = 'Helsinki-NLP/opus-mt-en-vi'

# Get the tokenizer
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained model based on the name
second_model = MarianMTModel.from_pretrained(second_model_name)

In [10]:
def format_batch_texts(language_code, batch_texts):
  formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]
  return formated_bach

def perform_translation(batch_texts, model, tokenizer, language="en"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

def perform_back_translation_with_augmentation_vi_en(batch_texts, original_language="vi", temporary_language="en"):
    # Define this shit
    batch_size = 50
    
    # Split the batch into smaller batches
    num_batches = len(batch_texts) // batch_size
    if len(batch_texts) % batch_size != 0:
        num_batches += 1

    # Initialize the final result
    final_result = []

    # Process each smaller batch
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        current_batch = batch_texts[start_idx:end_idx]

        # Translate from Original to Temporary Language
        tmp_translated_batch = perform_translation(current_batch, first_model, first_model_tkn, temporary_language)

        # Translate Back to English
        back_translated_batch = perform_translation(tmp_translated_batch, second_model, second_model_tkn, original_language)

        # Append the back-translated texts to the final result
        final_result.extend(back_translated_batch)

    # Return The Final Result
    return final_result

In [7]:
test4 = ["Người nghiện ma túy từ đủ 18 tuổi trở lên đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật xử lý vi phạm hành chính khi bị phát hiện sử dụng trái phép chất ma túy trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?"]
print(type(test4))
print(perform_back_translation_with_augmentation_vi_en(test4))
print(type(perform_back_translation_with_augmentation_vi_en(test4)))

<class 'list'>
['Phải chăng những người nghiện ma túy 18 tuổi và lớn tuổi bị bắt buộc phải bị điều trị bằng cách lạm dụng ma túy dưới sự cai trị của Luật pháp để lạm dụng ma túy khi lạm dụng ma túy, đúng hay không?']
<class 'list'>


In [8]:
original_text_list = test_df['original_text'].tolist()

In [11]:
marian_mt_vi_en = perform_back_translation_with_augmentation_vi_en(original_text_list)
marian_mt_vi_en

['NHỮNG người nghiện ngập 18 tuổi bị bắt buộc phải điều trị bằng cách lạm dụng ma túy dưới sự cai trị của Luật pháp khi người ta cho rằng ma túy bị lạm dụng khi lạm dụng ma túy trong việc lạm dụng ma túy, đúng hay không?',
 'Có mối quan hệ nào khác giữa hôn nhân và gia đình mà ít nhất một bên là người nước ngoài, định cư ở nước ngoài, có hay không?',
 'Có phải thông tin trong tập tin mật khẩu chứa thông tin nằm trong tập tin mật khẩu để tìm thông tin khác với cơ sở dữ liệu nhà chứa, dùng thông tin trong cơ sở dữ liệu nhà sở hữu, đúng hay sai?',
 'Theo luật du lịch, hợp đồng du lịch không cần phải được viết.',
 '"Sự giải quyết vĩnh viễn" là một phương pháp đối phó với những tranh chấp được thỏa thuận bởi các đảng phái và sau đó là các quy tắc của luật độc quyền.',
 'Thỏa thuận tranh cãi là vô ích khi phán quyết của tòa không có bất cứ hành vi dân sự nào phù hợp với luật pháp, đúng hay sai?',
 'Phải chăng người chồng phải hội đủ điều kiện để vợ được thiết lập, thực hành và chấm dứt các g

In [12]:
test_df['vi_en_marian_mt'] = marian_mt_vi_en

In [13]:
test_df

,original_text,vi2en,en2vi,vi_en_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ..."
...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?


In [14]:
# test_df.to_csv('output_2.csv', index=False, encoding='utf-8-sig')

### Tiếng Putin

In [16]:
# vi -> ru

# Get the name of the first model
third_model_name = 'Helsinki-NLP/opus-mt-vi-ru'
# Get the tokenizer
third_model_tkn = MarianTokenizer.from_pretrained(third_model_name)
# Load the pretrained model based on the name
third_model = MarianMTModel.from_pretrained(third_model_name)

In [17]:
# ru -> vi

# Get the name of the second model
fourth_model_name = 'Helsinki-NLP/opus-mt-ru-vi'

# Get the tokenizer
fourth_model_tkn = MarianTokenizer.from_pretrained(fourth_model_name)

# Load the pretrained model based on the name
fourth_model = MarianMTModel.from_pretrained(fourth_model_name)

In [18]:
def perform_translation2(batch_texts, model, tokenizer, language="ru"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

def perform_back_translation_with_augmentation_vi_ru(batch_texts, original_language="vi", temporary_language="ru"):
    # Define this shit
    batch_size = 100
    
    # Split the batch into smaller batches
    num_batches = len(batch_texts) // batch_size
    if len(batch_texts) % batch_size != 0:
        num_batches += 1

    # Initialize the final result
    final_result = []

    # Process each smaller batch
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        current_batch = batch_texts[start_idx:end_idx]

        # Translate from Original to Temporary Language
        tmp_translated_batch = perform_translation2(current_batch, third_model, third_model_tkn, temporary_language)

        # Translate Back to English
        back_translated_batch = perform_translation2(tmp_translated_batch, fourth_model, fourth_model_tkn, original_language)

        # Append the back-translated texts to the final result
        final_result.extend(back_translated_batch)

    # Return The Final Result
    return final_result

In [19]:
marian_mt_vi_ru = perform_back_translation_with_augmentation_vi_ru(original_text_list)
marian_mt_vi_ru

['▪ Nếu những người nghiện ma túy ở tuổi 18 trở thành những người nghiện ma túy, họ sẽ dùng một biện pháp quản trị có thể phục hồi chức năng phục hồi chức năng hoặc cai nghiện lạm dụng thuốc không đúng cách?',
 '▪ Hôn nhân và gia đình nước ngoài là hôn nhân và gia đình, ít nhất một trong số đó là người nước ngoài và người Việt Nam sống ở nước ngoài, có đúng không?',
 '> Thông tin trong sổ danh bạ mật mã có được dùng trong cơ sở dữ liệu cư trú hay không, như dữ liệu về nhà ở?',
 '[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] [ Nguồn tư liệu nơi trang 2]',
 '▪ Kinh doanh là một cách để giải quyết các vấn đề tranh chấp được đưa ra bởi các bên, và được thực hiện theo quy luật thương mại.',
 '▪ Nguồn tư liệu...',
 '● Người chồng có thể giao cho vợ giấy đăng ký, thực hiện và cắt đứt mọi giao dịch kinh doanh theo Luật Pháp về hôn nhân và đời sống gia đình không?',
 'LỢI ÍCH: Tôi bắt đầu tìm hiểu Kinh Thánh với Nhân Chứng Giê - hô - va.',
 'NẾU người hôn phối có quyền sở hữu trái đất mà họ có 

In [20]:
test_df['vi_ru_marian_mt'] = marian_mt_vi_ru
test_df

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_ru_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,▪ Nếu những người nghiện ma túy ở tuổi 18 trở ...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,▪ Hôn nhân và gia đình nước ngoài là hôn nhân ...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,> Thông tin trong sổ danh bạ mật mã có được dù...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...",[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] ...
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",▪ Kinh doanh là một cách để giải quyết các vấn...
...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,● trì hoãn bao lâu?
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Điều gì liên quan đến tài liệu được đề cập tro...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,"▪ Khi kết hôn, người nam và người nữ cần làm t..."
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,▪ [ Chú thích] Chúng ta có thể phân tích loại ...


In [21]:
# test_df.to_csv('output_3.csv', index=False, encoding='utf-8-sig')

### Tiếng Ra ta tu i

In [22]:
# vi -> fr

# Get the name of the first model
seventh_model_name = 'Helsinki-NLP/opus-mt-vi-fr'
# Get the tokenizer
seventh_model_tkn = MarianTokenizer.from_pretrained(seventh_model_name)
# Load the pretrained model based on the name
seventh_model = MarianMTModel.from_pretrained(seventh_model_name)

In [24]:
# fr -> vi

# Get the name of the second model
eighth_model_name = 'Helsinki-NLP/opus-mt-fr-vi'

# Get the tokenizer
eighth_model_tkn = MarianTokenizer.from_pretrained(eighth_model_name)

# Load the pretrained model based on the name
eighth_model = MarianMTModel.from_pretrained(eighth_model_name)

In [25]:
def perform_translation4(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

def perform_back_translation_with_augmentation_vi_fr(batch_texts, original_language="vi", temporary_language="fr"):
    # Define this shit
    batch_size = 100
    
    # Split the batch into smaller batches
    num_batches = len(batch_texts) // batch_size
    if len(batch_texts) % batch_size != 0:
        num_batches += 1

    # Initialize the final result
    final_result = []

    # Process each smaller batch
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        current_batch = batch_texts[start_idx:end_idx]

        # Translate from Original to Temporary Language
        tmp_translated_batch = perform_translation4(current_batch, seventh_model, seventh_model_tkn, temporary_language)

        # Translate Back to English
        back_translated_batch = perform_translation4(tmp_translated_batch, eighth_model, eighth_model_tkn, original_language)

        # Append the back-translated texts to the final result
        final_result.extend(back_translated_batch)

    # Return The Final Result
    return final_result

In [26]:
marian_mt_vi_fr = perform_back_translation_with_augmentation_vi_fr(original_text_list)
marian_mt_vi_fr

['Kể từ khi 18 tuổi, những người nghiện ma túy được áp dụng trong các trường cai nghiện mà Luật Pháp đòi hỏi phải điều chỉnh việc lạm dụng ma túy một cách bất hợp pháp trong thời kỳ cai nghiện, có hay không?',
 'Hôn nhân và gia đình có một yếu tố xa lạ: hôn nhân và gia đình, ít nhất một bên, là người lạ, người Việt Nam định cư ở nước ngoài, có hay không?',
 'Trong trường hợp thông tin trong két an toàn của một cổng truy cập không giống với dữ liệu nhập cư, dữ liệu trong cơ sở dữ liệu nhập cư, đúng hay sai?',
 'Theo luật đi du lịch, hợp đồng đó không cần phải được viết.',
 'Cụm từ "Log buôn bán" là giải pháp mà các bên ký kết và theo sau bởi luật pháp về dự luật thương mại.',
 'Hiệp ước đình chiến đã thất bại khi chúng ta thỏa thuận đình chiến mà không có quyền lực dân sự dựa trên luật hình sự, đúng hay sai?',
 'Người chồng có thể cho vợ quyền quyết định, áp dụng và chấm dứt cuộc trao đổi phù hợp với luật pháp về hôn nhân và gia đình không?',
 'Nếu một viên chức không hội đủ điều kiện, 

In [27]:
test_df['vi_fr_marian_mt'] = marian_mt_vi_fr
test_df

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_ru_marian_mt,vi_fr_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,▪ Nếu những người nghiện ma túy ở tuổi 18 trở ...,"Kể từ khi 18 tuổi, những người nghiện ma túy đ..."
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,▪ Hôn nhân và gia đình nước ngoài là hôn nhân ...,Hôn nhân và gia đình có một yếu tố xa lạ: hôn ...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,> Thông tin trong sổ danh bạ mật mã có được dù...,Trong trường hợp thông tin trong két an toàn c...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...",[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] ...,"Theo luật đi du lịch, hợp đồng đó không cần ph..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",▪ Kinh doanh là một cách để giải quyết các vấn...,"Cụm từ ""Log buôn bán"" là giải pháp mà các bên ..."
...,...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,● trì hoãn bao lâu?,Cần bao lâu để ngưng phục vụ nhà cầm quyền?
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Điều gì liên quan đến tài liệu được đề cập tro...,Hãy chọn câu trả lời tốt nhất.
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,"▪ Khi kết hôn, người nam và người nữ cần làm t...",Một người vợ nên sống theo tiêu chuẩn nào?
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,▪ [ Chú thích] Chúng ta có thể phân tích loại ...,Có phải xét nghiệm ma túy trong cơ thể là một ...


In [28]:
# test_df.to_csv('output_4.csv', index=False, encoding='utf-8-sig')

### Espanolllllllllllllllllllllll

In [29]:
# vi -> es

# Get the name of the first model
ninth_model_name = 'Helsinki-NLP/opus-mt-vi-es'
# Get the tokenizer
ninth_model_tkn = MarianTokenizer.from_pretrained(ninth_model_name)
# Load the pretrained model based on the name
ninth_model = MarianMTModel.from_pretrained(ninth_model_name)

In [30]:
# es -> vi

# Get the name of the second model
tenth_model_name = 'Helsinki-NLP/opus-mt-es-vi'

# Get the tokenizer
tenth_model_tkn = MarianTokenizer.from_pretrained(tenth_model_name)

# Load the pretrained model based on the name
tenth_model = MarianMTModel.from_pretrained(tenth_model_name)

In [33]:
def perform_translation5(batch_texts, model, tokenizer, language="es"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

def perform_back_translation_with_augmentation_vi_es(batch_texts, original_language="vi", temporary_language="es"):
    # Define this shit
    batch_size = 100
    
    # Split the batch into smaller batches
    num_batches = len(batch_texts) // batch_size
    if len(batch_texts) % batch_size != 0:
        num_batches += 1

    # Initialize the final result
    final_result = []

    # Process each smaller batch
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        current_batch = batch_texts[start_idx:end_idx]

        # Translate from Original to Temporary Language
        tmp_translated_batch = perform_translation5(current_batch, ninth_model, ninth_model_tkn, temporary_language)

        # Translate Back to English
        back_translated_batch = perform_translation5(tmp_translated_batch, tenth_model, tenth_model_tkn, original_language)

        # Append the back-translated texts to the final result
        final_result.extend(back_translated_batch)

    # Return The Final Result
    return final_result

In [34]:
marian_mt_vi_es = perform_back_translation_with_augmentation_vi_es(original_text_list)
marian_mt_vi_es

['Có đúng là những người nghiện ma túy trên 18 tuổi đã bị bắt buộc phải phục hồi chức năng phục hồi chức năng trong trường hợp bị phát hiện lạm dụng ma túy bất hợp pháp trong thời gian cai nghiện không?',
 'Phải chăng hôn nhân và gia đình là chuyện tình dục ngoài hôn nhân, và ít nhất một người là người ngoại quốc, người Việt Nam đã định cư ở nước ngoài?',
 'bối cảnh nào trong đó thông tin trong bản ghi có một thông tin khác với cơ sở dữ liệu cư trú, sử dụng thông tin trong cơ sở dữ liệu cư trú, có hay không?',
 'Theo Luật Báo chí, không cần phải ký hợp đồng du lịch.',
 'Tên Đồ Tể thương mại là giải pháp cho những cuộc xung đột đã được thỏa thuận và được thực hiện theo quy định của Đạo luật Súng Quốc Gia.',
 'Chẳng phải thỏa thuận của trọng tài không có tác dụng khi người ký hợp đồng với trọng tài không có khả năng cư xử công dân theo luật của Bộ Tư Pháp, đúng hay không?',
 'Người hôn phối trước có thể cho phép vợ mình thiết lập, thực hiện và chấm dứt các giao dịch theo luật hôn nhân và

In [35]:
test_df['vi_es_marian_mt'] = marian_mt_vi_es
test_df

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_ru_marian_mt,vi_fr_marian_mt,vi_es_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,▪ Nếu những người nghiện ma túy ở tuổi 18 trở ...,"Kể từ khi 18 tuổi, những người nghiện ma túy đ...",Có đúng là những người nghiện ma túy trên 18 t...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,▪ Hôn nhân và gia đình nước ngoài là hôn nhân ...,Hôn nhân và gia đình có một yếu tố xa lạ: hôn ...,Phải chăng hôn nhân và gia đình là chuyện tình...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,> Thông tin trong sổ danh bạ mật mã có được dù...,Trong trường hợp thông tin trong két an toàn c...,bối cảnh nào trong đó thông tin trong bản ghi ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...",[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] ...,"Theo luật đi du lịch, hợp đồng đó không cần ph...","Theo Luật Báo chí, không cần phải ký hợp đồng ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",▪ Kinh doanh là một cách để giải quyết các vấn...,"Cụm từ ""Log buôn bán"" là giải pháp mà các bên ...",Tên Đồ Tể thương mại là giải pháp cho những cu...
...,...,...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,● trì hoãn bao lâu?,Cần bao lâu để ngưng phục vụ nhà cầm quyền?,Giới hạn hoạt động của các viên chức đã bị đìn...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Điều gì liên quan đến tài liệu được đề cập tro...,Hãy chọn câu trả lời tốt nhất.,Đặc điểm mà thẻ hướng dẫn đề nghị bao gồm nhữn...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,"▪ Khi kết hôn, người nam và người nữ cần làm t...",Một người vợ nên sống theo tiêu chuẩn nào?,Những người nam và nữ lập gia đình phải hội đủ...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,▪ [ Chú thích] Chúng ta có thể phân tích loại ...,Có phải xét nghiệm ma túy trong cơ thể là một ...,Những loại thử nghiệm nào sau đây về ma túy tr...


In [36]:
test_df.to_csv('output_5.csv', index=False, encoding='utf-8-sig')

In [40]:
# ......

In [41]:
# Read the CSV file
test_df_2 = pd.read_csv('/kaggle/input/backtranslation-comp2/output_comp2_2.csv')
test_df_2

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_it_marian_mt,vi_de_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,65 phần trăm những người nghiện ma túy từ độ t...,Những kẻ nghiện ngập 18 tuổi bị đưa đến trại c...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,Có phải hôn nhân và gia đình là đặc điểm của n...,Mối quan hệ giữa vợ chồng và gia đình ở nước n...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,78% các trường hợp thông tin trong văn phòng c...,Nếu thông tin bạn tìm thấy trong thư điện tử k...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...","Theo Luật São Paulo, hợp đồng đi đường không n...","Theo luật thì không cần phải viết ra, đúng hay..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",Đó là cách mà các cuộc tranh luận được giải qu...,ADHS là một phương tiện để giải quyết các xung...
...,...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,Thời gian tối đa để đình chỉ công việc của các...,Thời gian tối đa cho phép tắt hệ thống chính p...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Dưới đây là những tài liệu nào đề nghị nên phâ...,Chúng ta phải làm gì trong hồ sơ của hướng dẫn...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,MERCUTIO Người nam và người nữ kết hôn trong n...,Những người kết hôn phải hội đủ những điều kiệ...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,Có những thử nghiệm nào sau đây để thử nghiệm ...,Việc kiểm tra ma túy trong cơ thể với những kỹ...


In [42]:
final_df = test_df
final_df

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_ru_marian_mt,vi_fr_marian_mt,vi_es_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,▪ Nếu những người nghiện ma túy ở tuổi 18 trở ...,"Kể từ khi 18 tuổi, những người nghiện ma túy đ...",Có đúng là những người nghiện ma túy trên 18 t...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,▪ Hôn nhân và gia đình nước ngoài là hôn nhân ...,Hôn nhân và gia đình có một yếu tố xa lạ: hôn ...,Phải chăng hôn nhân và gia đình là chuyện tình...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,> Thông tin trong sổ danh bạ mật mã có được dù...,Trong trường hợp thông tin trong két an toàn c...,bối cảnh nào trong đó thông tin trong bản ghi ...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...",[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] ...,"Theo luật đi du lịch, hợp đồng đó không cần ph...","Theo Luật Báo chí, không cần phải ký hợp đồng ..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",▪ Kinh doanh là một cách để giải quyết các vấn...,"Cụm từ ""Log buôn bán"" là giải pháp mà các bên ...",Tên Đồ Tể thương mại là giải pháp cho những cu...
...,...,...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,● trì hoãn bao lâu?,Cần bao lâu để ngưng phục vụ nhà cầm quyền?,Giới hạn hoạt động của các viên chức đã bị đìn...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Điều gì liên quan đến tài liệu được đề cập tro...,Hãy chọn câu trả lời tốt nhất.,Đặc điểm mà thẻ hướng dẫn đề nghị bao gồm nhữn...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,"▪ Khi kết hôn, người nam và người nữ cần làm t...",Một người vợ nên sống theo tiêu chuẩn nào?,Những người nam và nữ lập gia đình phải hội đủ...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,▪ [ Chú thích] Chúng ta có thể phân tích loại ...,Có phải xét nghiệm ma túy trong cơ thể là một ...,Những loại thử nghiệm nào sau đây về ma túy tr...


In [43]:
final_df['vi_it_marian_mt'] = test_df_2['vi_it_marian_mt']
final_df['vi_de_marian_mt'] = test_df_2['vi_de_marian_mt']
final_df

,original_text,vi2en,en2vi,vi_en_marian_mt,vi_ru_marian_mt,vi_fr_marian_mt,vi_es_marian_mt,vi_it_marian_mt,vi_de_marian_mt
0,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,Drug addicts aged full 18 years or older who a...,Người nghiện ma túy từ đủ 18 tuổi trở lên đang...,NHỮNG người nghiện ngập 18 tuổi bị bắt buộc ph...,▪ Nếu những người nghiện ma túy ở tuổi 18 trở ...,"Kể từ khi 18 tuổi, những người nghiện ma túy đ...",Có đúng là những người nghiện ma túy trên 18 t...,65 phần trăm những người nghiện ma túy từ độ t...,Những kẻ nghiện ngập 18 tuổi bị đưa đến trại c...
1,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Marriage and family relations involving foreig...,Quan hệ hôn nhân và gia đình có yếu tố nước ng...,Có mối quan hệ nào khác giữa hôn nhân và gia đ...,▪ Hôn nhân và gia đình nước ngoài là hôn nhân ...,Hôn nhân và gia đình có một yếu tố xa lạ: hôn ...,Phải chăng hôn nhân và gia đình là chuyện tình...,Có phải hôn nhân và gia đình là đặc điểm của n...,Mối quan hệ giữa vợ chồng và gia đình ở nước n...
2,Trường hợp thông tin trong Sổ hộ khẩu còn hiệu...,In case information in valid family registers ...,Trường hợp thông tin trong Sổ hộ khẩu còn giá ...,Có phải thông tin trong tập tin mật khẩu chứa ...,> Thông tin trong sổ danh bạ mật mã có được dù...,Trong trường hợp thông tin trong két an toàn c...,bối cảnh nào trong đó thông tin trong bản ghi ...,78% các trường hợp thông tin trong văn phòng c...,Nếu thông tin bạn tìm thấy trong thư điện tử k...
3,"Theo Luật Du Lịch, hợp đồng lữ hành không cần ...","According to the Tourism Law, travel contracts...","Theo Luật Du lịch, hợp đồng lữ hành không cần ...","Theo luật du lịch, hợp đồng du lịch không cần ...",[ Nguồn tư liệu nơi trang 2] [ Nguồn tư liệu] ...,"Theo luật đi du lịch, hợp đồng đó không cần ph...","Theo Luật Báo chí, không cần phải ký hợp đồng ...","Theo Luật São Paulo, hợp đồng đi đường không n...","Theo luật thì không cần phải viết ra, đúng hay..."
4,"""Trọng tại thương mại"" là phương thức giải quy...","""Commercial Arbitration"" refers to a method of...","""Trọng tài thương mại"" là phương thức giải quy...","""Sự giải quyết vĩnh viễn"" là một phương pháp đ...",▪ Kinh doanh là một cách để giải quyết các vấn...,"Cụm từ ""Log buôn bán"" là giải pháp mà các bên ...",Tên Đồ Tể thương mại là giải pháp cho những cu...,Đó là cách mà các cuộc tranh luận được giải qu...,ADHS là một phương tiện để giải quyết các xung...
...,...,...,...,...,...,...,...,...,...
3911,Thời hạn tối đa tạm đình chỉ công tác đối với ...,How long is the maximum time limit for suspens...,Thời gian đình chỉ công chức tối đa là bao lâu?,Thời gian nhất định phải làm việc chung với vi...,● trì hoãn bao lâu?,Cần bao lâu để ngưng phục vụ nhà cầm quyền?,Giới hạn hoạt động của các viên chức đã bị đìn...,Thời gian tối đa để đình chỉ công việc của các...,Thời gian tối đa cho phép tắt hệ thống chính p...
3912,Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lị...,What is the following information to be includ...,Những thông tin sau đây cần có trong hồ sơ đề ...,Hãy chọn câu trả lời tốt nhất.,Điều gì liên quan đến tài liệu được đề cập tro...,Hãy chọn câu trả lời tốt nhất.,Đặc điểm mà thẻ hướng dẫn đề nghị bao gồm nhữn...,Dưới đây là những tài liệu nào đề nghị nên phâ...,Chúng ta phải làm gì trong hồ sơ của hướng dẫn...
3913,Nam nữ kết hôn với nhau phải tuân theo các điề...,In which of the following conditions must be s...,Trong những điều kiện sau đây phải đáp ứng cho...,Các tình trạng sau khi các điều điều lệ phải đ...,"▪ Khi kết hôn, người nam và người nữ cần làm t...",Một người vợ nên sống theo tiêu chuẩn nào?,Những người nam và nữ lập gia đình phải hội đủ...,MERCUTIO Người nam và người nữ kết hôn trong n...,Những người kết hôn phải hội đủ những điều kiệ...
3914,Xét nghiệm chất ma túy trong cơ thể là việc th...,Drug Testing is the practice of performing the...,Kiểm nghiệm thuốc là việc thực hành các kỹ thu...,Một số mẫu thử nghiệm trong cơ thể sau đây là gì?,▪ [ Chú thích] Chúng ta có thể phân tích loại ...,

In [44]:
final_df.to_csv('final_backtranslation_1.csv', index=False, encoding='utf-8-sig')